In [1]:
import pandas as pd
import re
import pymongo

In [2]:
# Kết nối tới MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")  # Thay thế bằng địa chỉ MongoDB của bạn
db = client["TGDD_CRAWLER"]  # Tên database của bạn
collection = db["DHDT_Collection"]  # Tên collection chứa dữ liệu

In [10]:
# Trích xuất dữ liệu từ MongoDB và bỏ qua trường '_id'
cursor = collection.find({}, {'_id': 0})

# Chuyển dữ liệu từ MongoDB sang DataFrame của Pandas
df = pd.DataFrame(list(cursor))

# Hiển thị 5 dòng đầu tiên
df.head()

,Name,Price,Old Price,Promotion,Thời gian sử dụng pin:,Đối tượng sử dụng:,Đường kính mặt:,Độ dày mặt:,Kháng nước:,Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Bình luận
0,SMILE KID 38 mm Trẻ em SL414-010SK,290000.0,390000.0,"Nhập mã VNPAYTGDD4 giảm từ 50,000đ đến 100,000...",Khoảng 1.5 năm,Trẻ em,38 mm,14 mm,"5 ATM - Đi mưa, tắm",Pin,Nhựa Resin,Việt Nam,N/A
1,CASIO 42 mm Nam AE-1200WH-1AVDF,820000.0,1259000.0,"Nhập mã VNPAYTGDD4 giảm từ 50,000đ đến 100,000...",Hãng không công bố,Nam,42 mm,Hãng không công bố,"10 ATM - Tắm, bơi",Pin,Nhựa,Nhật Bản,Cháu đeo đi học rất đúng giờ
2,MVW Glory 43 mm Nam ML080-01,1290000.0,2250000.0,"Nhập mã VNPAYTGDD4 giảm từ 50,000đ đến 100,000...",Khoảng 3 năm,Nam,43 mm,9.2 mm,"5 ATM - Đi mưa, tắm",Pin,Kính khoáng Mineral,Việt Nam,Mình cần mua dây đồng hồ này có bán ko
3,CASIO 43 mm Nam AE-1000W-2AVDF,852000.0,1136000.0,"Nhập mã VNPAYTGDD4 giảm từ 50,000đ đến 100,000...",Khoảng 10 năm,Nam,43 mm,13.7 mm,"10 ATM - Tắm, bơi",Pin,Nhựa,Nhật Bản,Hài lòng
4,MVW Sport 43 mm Nam ML074-01,990000.0,1990000.0,"Nhập mã VNPAYTGDD4 giảm từ 50,000đ đến 100,000...",Khoảng 3 năm,Nam,43 mm,10.3 mm,"5 ATM - Đi mưa, tắm",Pin,Kính khoáng Mineral,Việt Nam,"Mình bị đứt dây, có được hor trợ bảo hành thay..."


In [11]:
def process_name(name):
    # Loại bỏ từ "Mẫu mới" nếu có
    name = name.replace("Mẫu mới", "").strip()
    
    # Loại bỏ các chuỗi kích thước như 'xx mm', 'xx x yy mm', 'xx.x mm', hoặc 'xx × yy mm' và các thông tin giới tính
    clean_name = re.sub(r'\d+(\.\d+)?( x \d+(\.\d+)?| × \d+(\.\d+)?)*\s*mm|Nam|Nữ|Trẻ em|Unisex', '', name).strip()

    # Loại bỏ dấu gạch nối hoặc khoảng trắng không cần thiết
    clean_name = re.sub(r'\s*-\s*', '', clean_name).strip()

    # Loại bỏ từ "Eco-Drive" nếu có ở cuối chuỗi
    clean_name = re.sub(r'Eco-Drive$', '', clean_name).strip()

    # Tìm mã sản phẩm dựa trên định dạng mã (các chữ và số cuối)
    product_code = re.search(r'[A-Z0-9\/\-\.]+$', clean_name)
    
    # Tách tên sản phẩm và mã sản phẩm
    if product_code:
        name_only = clean_name.replace(product_code.group(), '').strip()
        code = product_code.group()
    else:
        name_only = clean_name
        code = "Unknown"
    
    return name_only, code

# Áp dụng hàm xử lý cho cột 'Name'
df['Tên sản phẩm'], df['Mã sản phẩm'] = zip(*df['Name'].apply(process_name))

# Đổi vị trí các cột nếu cần thiết (thêm vào đầu hoặc cuối DataFrame)
df = df[['Tên sản phẩm', 'Mã sản phẩm'] + [col for col in df.columns if col not in ['Tên sản phẩm', 'Mã sản phẩm']]]

In [12]:
# Xóa cột 'Name' khỏi dataframe
df.drop(columns=['Name'], inplace=True)

In [13]:
df.head(5)

,Tên sản phẩm,Mã sản phẩm,Price,Old Price,Promotion,Thời gian sử dụng pin:,Đối tượng sử dụng:,Đường kính mặt:,Độ dày mặt:,Kháng nước:,Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Bình luận
0,SMILE KID,SL414010SK,290000.0,390000.0,"Nhập mã VNPAYTGDD4 giảm từ 50,000đ đến 100,000...",Khoảng 1.5 năm,Trẻ em,38 mm,14 mm,"5 ATM - Đi mưa, tắm",Pin,Nhựa Resin,Việt Nam,N/A
1,CASIO,AE1200WH1AVDF,820000.0,1259000.0,"Nhập mã VNPAYTGDD4 giảm từ 50,000đ đến 100,000...",Hãng không công bố,Nam,42 mm,Hãng không công bố,"10 ATM - Tắm, bơi",Pin,Nhựa,Nhật Bản,Cháu đeo đi học rất đúng giờ
2,MVW Glory,ML08001,1290000.0,2250000.0,"Nhập mã VNPAYTGDD4 giảm từ 50,000đ đến 100,000...",Khoảng 3 năm,Nam,43 mm,9.2 mm,"5 ATM - Đi mưa, tắm",Pin,Kính khoáng Mineral,Việt Nam,Mình cần mua dây đồng hồ này có bán ko
3,CASIO,AE1000W2AVDF,852000.0,1136000.0,"Nhập mã VNPAYTGDD4 giảm từ 50,000đ đến 100,000...",Khoảng 10 năm,Nam,43 mm,13.7 mm,"10 ATM - Tắm, bơi",Pin,Nhựa,Nhật Bản,Hài lòng
4,MVW Sport,ML07401,990000.0,1990000.0,"Nhập mã VNPAYTGDD4 giảm từ 50,000đ đến 100,000...",Khoảng 3 năm,Nam,43 mm,10.3 mm,"5 ATM - Đi mưa, tắm",Pin,Kính khoáng Mineral,Việt Nam,"Mình bị đứt dây, có được hor trợ bảo hành thay..."


In [14]:
# Hàm xử lý cột 'Promotion'
def process_promotion(promotion):
    # Tìm mã giảm giá
    discount_code = re.search(r'mã (\w+)', promotion)
    discount_code = discount_code.group(1) if discount_code else None

    # Tìm mức giảm tối thiểu và tối đa
    min_discount = re.search(r'giảm từ (\d{1,3}(?:,\d{3})*|\d+)', promotion)
    max_discount = re.search(r'đến (\d{1,3}(?:,\d{3})*|\d+)', promotion)

    # Chuyển đổi thành số nguyên
    min_discount = int(min_discount.group(1).replace(',', '')) if min_discount else None
    max_discount = int(max_discount.group(1).replace(',', '')) if max_discount else None

    # Tìm phương thức thanh toán khuyến mãi
    payment_method = re.search(r'thanh toán qua (\w+-\w+)', promotion)
    payment_method = payment_method.group(1) if payment_method else None
    
    return discount_code, min_discount, max_discount, payment_method

# Áp dụng hàm xử lý cho cột 'Promotion' để tạo các cột mới
df['Mã giảm giá'], df['Mức giảm tối thiểu'], df['Mức giảm tối đa'], df['Phương thức thanh toán nhận khuyến mãi'] = zip(*df['Promotion'].apply(process_promotion))



In [15]:
df.drop(columns="Promotion", inplace=True)

In [16]:
# Đưa các cột mới vào vị trí trước cột 'Old Price'
cols = df.columns.tolist()
new_order = cols[:2] + cols[-4:] + cols[2:-4]  # Đưa các cột mới vào vị trí trước cột 'Old Price'
df = df[new_order]

In [17]:
df.head(3)

,Tên sản phẩm,Mã sản phẩm,Mã giảm giá,Mức giảm tối thiểu,Mức giảm tối đa,Phương thức thanh toán nhận khuyến mãi,Price,Old Price,Thời gian sử dụng pin:,Đối tượng sử dụng:,Đường kính mặt:,Độ dày mặt:,Kháng nước:,Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Bình luận
0,SMILE KID,SL414010SK,VNPAYTGDD4,50000,100000,VNPAY-QR,290000.0,390000.0,Khoảng 1.5 năm,Trẻ em,38 mm,14 mm,"5 ATM - Đi mưa, tắm",Pin,Nhựa Resin,Việt Nam,N/A
1,CASIO,AE1200WH1AVDF,VNPAYTGDD4,50000,100000,VNPAY-QR,820000.0,1259000.0,Hãng không công bố,Nam,42 mm,Hãng không công bố,"10 ATM - Tắm, bơi",Pin,Nhựa,Nhật Bản,Cháu đeo đi học rất đúng giờ
2,MVW Glory,ML08001,VNPAYTGDD4,50000,100000,VNPAY-QR,1290000.0,2250000.0,Khoảng 3 năm,Nam,43 mm,9.2 mm,"5 ATM - Đi mưa, tắm",Pin,Kính khoáng Mineral,Việt Nam,Mình cần mua dây đồng hồ này có bán ko


In [18]:
# Hàm xử lý thời gian sử dụng pin
def process_battery_life(battery_life):
    if pd.isna(battery_life) or battery_life == '':
        return float('nan')  # Trả về NaN cho giá trị rỗng hoặc None
    
    months = 0  # Khởi tạo biến lưu số tháng
    if 'tháng' in battery_life:
        months_match = re.search(r'(\d+(\.\d+)?)', battery_life)  # Tìm số tháng
        if months_match:
            months += float(months_match.group(1))  # Cộng thêm số tháng (có thể là số thập phân)
    elif 'năm' in battery_life:
        years_match = re.search(r'(\d+(\.\d+)?)', battery_life)  # Tìm số năm
        if years_match:
            months += float(years_match.group(1)) * 12  # Chuyển đổi năm thành tháng
    
    return months  # Trả về số tháng dưới dạng float

# Xử lý cột 'Thời gian sử dụng pin'
df['Thời gian sử dụng pin:'] = df['Thời gian sử dụng pin:'].apply(process_battery_life)

In [19]:
# Đổi tên cột thành "Thời gian sử dụng pin (năm)"
df.rename(columns={'Thời gian sử dụng pin:': 'Thời gian sử dụng pin (tháng)'}, inplace=True)

In [20]:
df.head()

,Tên sản phẩm,Mã sản phẩm,Mã giảm giá,Mức giảm tối thiểu,Mức giảm tối đa,Phương thức thanh toán nhận khuyến mãi,Price,Old Price,Thời gian sử dụng pin (tháng),Đối tượng sử dụng:,Đường kính mặt:,Độ dày mặt:,Kháng nước:,Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Bình luận
0,SMILE KID,SL414010SK,VNPAYTGDD4,50000,100000,VNPAY-QR,290000.0,390000.0,18.0,Trẻ em,38 mm,14 mm,"5 ATM - Đi mưa, tắm",Pin,Nhựa Resin,Việt Nam,N/A
1,CASIO,AE1200WH1AVDF,VNPAYTGDD4,50000,100000,VNPAY-QR,820000.0,1259000.0,0.0,Nam,42 mm,Hãng không công bố,"10 ATM - Tắm, bơi",Pin,Nhựa,Nhật Bản,Cháu đeo đi học rất đúng giờ
2,MVW Glory,ML08001,VNPAYTGDD4,50000,100000,VNPAY-QR,1290000.0,2250000.0,36.0,Nam,43 mm,9.2 mm,"5 ATM - Đi mưa, tắm",Pin,Kính khoáng Mineral,Việt Nam,Mình cần mua dây đồng hồ này có bán ko
3,CASIO,AE1000W2AVDF,VNPAYTGDD4,50000,100000,VNPAY-QR,852000.0,1136000.0,120.0,Nam,43 mm,13.7 mm,"10 ATM - Tắm, bơi",Pin,Nhựa,Nhật Bản,Hài lòng
4,MVW Sport,ML07401,VNPAYTGDD4,50000,100000,VNPAY-QR,990000.0,1990000.0,36.0,Nam,43 mm,10.3 mm,"5 ATM - Đi mưa, tắm",Pin,Kính khoáng Mineral,Việt Nam,"Mình bị đứt dây, có được hor trợ bảo hành thay..."


In [21]:
df.head()

,Tên sản phẩm,Mã sản phẩm,Mã giảm giá,Mức giảm tối thiểu,Mức giảm tối đa,Phương thức thanh toán nhận khuyến mãi,Price,Old Price,Thời gian sử dụng pin (tháng),Đối tượng sử dụng:,Đường kính mặt:,Độ dày mặt:,Kháng nước:,Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Bình luận
0,SMILE KID,SL414010SK,VNPAYTGDD4,50000,100000,VNPAY-QR,290000.0,390000.0,18.0,Trẻ em,38 mm,14 mm,"5 ATM - Đi mưa, tắm",Pin,Nhựa Resin,Việt Nam,N/A
1,CASIO,AE1200WH1AVDF,VNPAYTGDD4,50000,100000,VNPAY-QR,820000.0,1259000.0,0.0,Nam,42 mm,Hãng không công bố,"10 ATM - Tắm, bơi",Pin,Nhựa,Nhật Bản,Cháu đeo đi học rất đúng giờ
2,MVW Glory,ML08001,VNPAYTGDD4,50000,100000,VNPAY-QR,1290000.0,2250000.0,36.0,Nam,43 mm,9.2 mm,"5 ATM - Đi mưa, tắm",Pin,Kính khoáng Mineral,Việt Nam,Mình cần mua dây đồng hồ này có bán ko
3,CASIO,AE1000W2AVDF,VNPAYTGDD4,50000,100000,VNPAY-QR,852000.0,1136000.0,120.0,Nam,43 mm,13.7 mm,"10 ATM - Tắm, bơi",Pin,Nhựa,Nhật Bản,Hài lòng
4,MVW Sport,ML07401,VNPAYTGDD4,50000,100000,VNPAY-QR,990000.0,1990000.0,36.0,Nam,43 mm,10.3 mm,"5 ATM - Đi mưa, tắm",Pin,Kính khoáng Mineral,Việt Nam,"Mình bị đứt dây, có được hor trợ bảo hành thay..."


In [22]:
# Hàm xử lý cột 'Kháng nước:'
def process_water_resistance(resistance):
    if pd.isna(resistance):  # Kiểm tra giá trị NaN
        return None
    
    match = re.match(r'(\d+)\s*ATM\s*-\s*(.*)', resistance)
    if match:
        level = int(match.group(1))  # Chuyển đổi mức kháng nước thành số
        return level
    return None

# Áp dụng hàm xử lý cho cột 'Kháng nước:'
df['Mức kháng nước']= df['Kháng nước:'].apply(process_water_resistance)

# Đổi tên cột 'Mức kháng nước' để thêm đơn vị 'ATM'
df.rename(columns={'Mức kháng nước': 'Mức kháng nước (ATM)'}, inplace=True)

# Xóa cột 'Kháng nước:' khỏi DataFrame
df.drop(columns=['Kháng nước:'], inplace=True)

In [23]:
df.head()

,Tên sản phẩm,Mã sản phẩm,Mã giảm giá,Mức giảm tối thiểu,Mức giảm tối đa,Phương thức thanh toán nhận khuyến mãi,Price,Old Price,Thời gian sử dụng pin (tháng),Đối tượng sử dụng:,Đường kính mặt:,Độ dày mặt:,Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Bình luận,Mức kháng nước (ATM)
0,SMILE KID,SL414010SK,VNPAYTGDD4,50000,100000,VNPAY-QR,290000.0,390000.0,18.0,Trẻ em,38 mm,14 mm,Pin,Nhựa Resin,Việt Nam,N/A,5.0
1,CASIO,AE1200WH1AVDF,VNPAYTGDD4,50000,100000,VNPAY-QR,820000.0,1259000.0,0.0,Nam,42 mm,Hãng không công bố,Pin,Nhựa,Nhật Bản,Cháu đeo đi học rất đúng giờ,10.0
2,MVW Glory,ML08001,VNPAYTGDD4,50000,100000,VNPAY-QR,1290000.0,2250000.0,36.0,Nam,43 mm,9.2 mm,Pin,Kính khoáng Mineral,Việt Nam,Mình cần mua dây đồng hồ này có bán ko,5.0
3,CASIO,AE1000W2AVDF,VNPAYTGDD4,50000,100000,VNPAY-QR,852000.0,1136000.0,120.0,Nam,43 mm,13.7 mm,Pin,Nhựa,Nhật Bản,Hài lòng,10.0
4,MVW Sport,ML07401,VNPAYTGDD4,50000,100000,VNPAY-QR,990000.0,1990000.0,36.0,Nam,43 mm,10.3 mm,Pin,Kính khoáng Mineral,Việt Nam,"Mình bị đứt dây, có được hor trợ bảo hành thay...",5.0


In [24]:
# Hàm xử lý cho cột 'Đường kính mặt:'
def process_diameter(diameter):
    if pd.isna(diameter) or diameter == 'N/A':
        return None  # Trả về None cho giá trị NaN hoặc 'N/A'
    if isinstance(diameter, str):  # Kiểm tra xem có phải là chuỗi không
        # Tìm tất cả các số và mm
        match = re.match(r'(\d+(\.\d+)?)\s*mm', diameter)
        if match:
            return float(match.group(1))  # Trả về giá trị đầu tiên
        # Nếu có định dạng như "xx x yy mm", ta có thể lấy giá trị đầu tiên
        match_dimension = re.search(r'(\d+(\.\d+)?)\s*x\s*(\d+(\.\d+)?)\s*mm', diameter)
        if match_dimension:
            return float(match_dimension.group(1))  # Trả về giá trị đầu tiên
    return None  # Trả về None nếu không tìm thấy

# Hàm xử lý cho cột 'Độ dày mặt:'
def process_thickness(thickness):
    if pd.isna(thickness) or thickness == 'N/A':
        return None  # Trả về None cho giá trị NaN hoặc 'N/A'
    if isinstance(thickness, str):  # Kiểm tra xem có phải là chuỗi không
        match = re.match(r'(\d+(\.\d+)?)\s*mm', thickness)
        if match:
            return float(match.group(1))  # Chuyển đổi thành số thực
    return None  # Trả về None cho các giá trị không hợp lệ

# Ghi đè lên cột cũ với giá trị đã xử lý
df['Đường kính mặt:'] = df['Đường kính mặt:'].apply(process_diameter)
df['Độ dày mặt:'] = df['Độ dày mặt:'].apply(process_thickness)

In [25]:
# Đổi tên các cột với đơn vị trong ngoặc
df.rename(columns={
    'Đường kính mặt:': 'Đường kính mặt (mm)',
    'Độ dày mặt:': 'Độ dày mặt (mm)'
}, inplace=True)

In [26]:
df.head()

,Tên sản phẩm,Mã sản phẩm,Mã giảm giá,Mức giảm tối thiểu,Mức giảm tối đa,Phương thức thanh toán nhận khuyến mãi,Price,Old Price,Thời gian sử dụng pin (tháng),Đối tượng sử dụng:,Đường kính mặt (mm),Độ dày mặt (mm),Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Bình luận,Mức kháng nước (ATM)
0,SMILE KID,SL414010SK,VNPAYTGDD4,50000,100000,VNPAY-QR,290000.0,390000.0,18.0,Trẻ em,38.0,14.0,Pin,Nhựa Resin,Việt Nam,N/A,5.0
1,CASIO,AE1200WH1AVDF,VNPAYTGDD4,50000,100000,VNPAY-QR,820000.0,1259000.0,0.0,Nam,42.0,NaN,Pin,Nhựa,Nhật Bản,Cháu đeo đi học rất đúng giờ,10.0
2,MVW Glory,ML08001,VNPAYTGDD4,50000,100000,VNPAY-QR,1290000.0,2250000.0,36.0,Nam,43.0,9.2,Pin,Kính khoáng Mineral,Việt Nam,Mình cần mua dây đồng hồ này có bán ko,5.0
3,CASIO,AE1000W2AVDF,VNPAYTGDD4,50000,100000,VNPAY-QR,852000.0,1136000.0,120.0,Nam,43.0,13.7,Pin,Nhựa,Nhật Bản,Hài lòng,10.0
4,MVW Sport,ML07401,VNPAYTGDD4,50000,100000,VNPAY-QR,990000.0,1990000.0,36.0,Nam,43.0,10.3,Pin,Kính khoáng Mineral,Việt Nam,"Mình bị đứt dây, có được hor trợ bảo hành thay...",5.0


In [27]:
# Thay thế các bình luận rỗng hoặc không ghi gì bằng "N/A"
df['Bình luận'] = df['Bình luận'].replace('', 'N/A')  # Thay thế chuỗi rỗng
df['Bình luận'] = df['Bình luận'].fillna('N/A')       # Thay thế giá trị None
df.head(20)

,Tên sản phẩm,Mã sản phẩm,Mã giảm giá,Mức giảm tối thiểu,Mức giảm tối đa,Phương thức thanh toán nhận khuyến mãi,Price,Old Price,Thời gian sử dụng pin (tháng),Đối tượng sử dụng:,Đường kính mặt (mm),Độ dày mặt (mm),Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Bình luận,Mức kháng nước (ATM)
0,SMILE KID,SL414010SK,VNPAYTGDD4,50000,100000,VNPAY-QR,290000.0,390000.0,18.0,Trẻ em,38.0,14.0,Pin,Nhựa Resin,Việt Nam,N/A,5.0
1,CASIO,AE1200WH1AVDF,VNPAYTGDD4,50000,100000,VNPAY-QR,820000.0,1259000.0,0.0,Nam,42.0,NaN,Pin,Nhựa,Nhật Bản,Cháu đeo đi học rất đúng giờ,10.0
2,MVW Glory,ML08001,VNPAYTGDD4,50000,100000,VNPAY-QR,1290000.0,2250000.0,36.0,Nam,43.0,9.2,Pin,Kính khoáng Mineral,Việt Nam,Mình cần mua dây đồng hồ này có bán ko,5.0
3,CASIO,AE1000W2AVDF,VNPAYTGDD4,50000,100000,VNPAY-QR,852000.0,1136000.0,120.0,Nam,43.0,13.7,Pin,Nhựa,Nhật Bản,Hài lòng,10.0
4,MVW Sport,ML07401,VNPAYTGDD4,50000,100000,VNPAY-QR,990000.0,1990000.0,36.0,Nam,43.0,10.3,Pin,Kính khoáng Mineral,Việt Nam,"Mình bị đứt dây, có được hor trợ bảo hành thay...",5.0
5,CASIO,LTPV300L7A2UDF,VNPAYTGDD4,50000,100000,VNPAY-QR,1138000.0,1752000.0,24.0,Nữ,33.2,8.2,Pin,Kính khoáng Mineral,Nhật Bản,"Mặt đồng hồ rất đẹp, dây da màu cà phê sữa ko ...",3.0
6,ELIO Nắng Xuân,ES06102,VNPAYTGDD4,50000,100000,VNPAY-QR,690000.0,1290000.0,36.0,Nữ,30.0,8.0,Pin,Kính khoáng Mineral,Việt Nam,Rất tốt,3.0
7,ELIO,ES015C2,VNPAYTGDD4,50000,100000,VNPAY-QR,390000.0,1190000.0,12.0,Nữ,29.5,7.5,Pin,Kính khoáng Mineral,Việt Nam,Nói chung sử dụng tốt và bền,3.0
8,CASIO,LTPV005L7AUDF,VNPAYTGDD4,50000,100000,VNPAY-QR,509000.0,784000.0,24.0,Nữ,28.2,7.9,Pin,Kính khoáng Mineral,Nhật Bản,Tốt,3.0
9,SMILE KID,SL02301,VNPAYTGDD4,50000,100000,VNPAY-QR,290000.0,350000.0,18.0,Trẻ em,37.0,14.0,Pin,Nhựa Resin,Việt Nam,Đồng hồ rất đẹp,5.0


In [28]:
# Thứ tự cột mong muốn
new_order = [
    'Tên sản phẩm', 'Mã sản phẩm', 'Mã giảm giá', 
    'Mức giảm tối thiểu', 'Mức giảm tối đa', 
    'Phương thức thanh toán nhận khuyến mãi', 
    'Price', 'Old Price', 'Thời gian sử dụng pin (tháng)', 
    'Đối tượng sử dụng:', 'Đường kính mặt (mm)', 
    'Độ dày mặt (mm)', 'Nguồn năng lượng:', 
    'Mức kháng nước (ATM)', 
    'Chất liệu mặt kính:', 'Thương hiệu của:', 
    'Bình luận'
]

# Sắp xếp lại cột
df = df[new_order]
df.head()

,Tên sản phẩm,Mã sản phẩm,Mã giảm giá,Mức giảm tối thiểu,Mức giảm tối đa,Phương thức thanh toán nhận khuyến mãi,Price,Old Price,Thời gian sử dụng pin (tháng),Đối tượng sử dụng:,Đường kính mặt (mm),Độ dày mặt (mm),Nguồn năng lượng:,Mức kháng nước (ATM),Chất liệu mặt kính:,Thương hiệu của:,Bình luận
0,SMILE KID,SL414010SK,VNPAYTGDD4,50000,100000,VNPAY-QR,290000.0,390000.0,18.0,Trẻ em,38.0,14.0,Pin,5.0,Nhựa Resin,Việt Nam,N/A
1,CASIO,AE1200WH1AVDF,VNPAYTGDD4,50000,100000,VNPAY-QR,820000.0,1259000.0,0.0,Nam,42.0,NaN,Pin,10.0,Nhựa,Nhật Bản,Cháu đeo đi học rất đúng giờ
2,MVW Glory,ML08001,VNPAYTGDD4,50000,100000,VNPAY-QR,1290000.0,2250000.0,36.0,Nam,43.0,9.2,Pin,5.0,Kính khoáng Mineral,Việt Nam,Mình cần mua dây đồng hồ này có bán ko
3,CASIO,AE1000W2AVDF,VNPAYTGDD4,50000,100000,VNPAY-QR,852000.0,1136000.0,120.0,Nam,43.0,13.7,Pin,10.0,Nhựa,Nhật Bản,Hài lòng
4,MVW Sport,ML07401,VNPAYTGDD4,50000,100000,VNPAY-QR,990000.0,1990000.0,36.0,Nam,43.0,10.3,Pin,5.0,Kính khoáng Mineral,Việt Nam,"Mình bị đứt dây, có được hor trợ bảo hành thay..."


In [29]:
# Xử lý tất cả các cột: thay thế các giá trị rỗng hoặc None bằng "N/A"
df.fillna('N/A', inplace=True)  # Thay thế giá trị None
df.replace('', 'N/A', inplace=True)  # Thay thế chuỗi rỗng

C:\Users\T&T\AppData\Local\Temp\ipykernel_9840\1666721214.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('N/A', inplace=True)  # Thay thế giá trị None


In [30]:
# Lưu DataFrame vào file Excel mới
df.to_excel('processed_data.xlsx', index=False)

print("Dữ liệu đã được lưu vào file 'processed_data.xlsx'.")

Dữ liệu đã được lưu vào file 'processed_data.xlsx'.


In [31]:
import pandas as pd
import json

# Giả sử bạn đã có DataFrame 'df' đã được xử lý

# Lưu dữ liệu vào file JSON
with open('DHDT_json_Processed.json', 'w', encoding='utf-8') as json_file:
    json.dump(df.to_dict(orient='records'), json_file, ensure_ascii=False, indent=4)

# Lưu DataFrame ra định dạng CSV
df.to_csv('DHDT_csv_Processed.csv', index=False, encoding='utf-8')

print("Dữ liệu đã được lưu vào file JSON và CSV.")


Dữ liệu đã được lưu vào file JSON và CSV.
